# Classifier Evaluation

In this notebook we test a bunch of different classifiers in order to find the 3 best performing one. Those three will then be combined in an ensemble model to form our final classifier.

In [1]:
%load_ext autoreload
%autoreload 2
%matplotlib inline

In [2]:
import os
import sys
sys.path.append('./lib')

In [3]:
import numpy as np
import pandas as pd

import time
from pathlib import Path

from sklearn.metrics import log_loss, classification_report, accuracy_score

from tqdm import tqdm_notebook as tqdm

In [4]:
from lib.definitions import RANDOM_SEED

In [5]:
import random
random.seed(RANDOM_SEED)

import numpy.random
numpy.random.seed(RANDOM_SEED)

import os
os.environ['PYTHONHASHSEED']=str(RANDOM_SEED)

import tensorflow
tensorflow.set_random_seed(RANDOM_SEED)

In [6]:
from lib.definitions import SPLITS_BASE_PATH

SPLITS_BASE_PATH.mkdir(parents=True, exist_ok=True)

## Find the top 3 best performing classifiers

In [7]:
from lib.dataset import load_split, evaluate_estimator, evaluate_splits
from lib.classifiers import classifier_factories

results = {}

for (name, factory) in classifier_factories.items():
    t0 = time.time()
    print(f'Evaluating "{name}"')
    scores = evaluate_splits(factory)
    duration = time.time() - t0
    print(f'[Took {duration}s]\n')
    
    results[name] = {
        'scores': scores,
        'duration': duration
    }

Evaluating "Random Forest"
[Took 0.9301531314849854s]

Evaluating "XGBoost"
[Took 0.10933399200439453s]

Evaluating "K-Nearest Neighbour"
[Took 0.38782691955566406s]

Evaluating "Naive Bayes"
[Took 0.04604482650756836s]

Evaluating "Quadratic Discriminant Analysis"
[Took 0.05256295204162598s]

Evaluating "Adaptive Boosting"
[Took 0.24199175834655762s]

Evaluating "Multilayer Perceptron"


/Users/marc/miniconda2/envs/xai_test/lib/python3.6/site-packages/sklearn/neural_network/multilayer_perceptron.py:566: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  % self.max_iter, ConvergenceWarning)
/Users/marc/miniconda2/envs/xai_test/lib/python3.6/site-packages/sklearn/neural_network/multilayer_perceptron.py:566: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  % self.max_iter, ConvergenceWarning)


[Took 0.45665717124938965s]

Evaluating "Deep Neural Network"
Instructions for updating:
Colocations handled automatically by placer.


/Users/marc/miniconda2/envs/xai_test/lib/python3.6/site-packages/sklearn/neural_network/multilayer_perceptron.py:566: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  % self.max_iter, ConvergenceWarning)
Using TensorFlow backend.


Instructions for updating:
Use tf.cast instead.
[Took 2.670112133026123s]



In [8]:
results

{'Random Forest': {'scores': [0.7888888888888889,
   0.8111111111111111,
   0.8777777777777778],
  'duration': 0.9301531314849854},
 'XGBoost': {'scores': [0.7444444444444445,
   0.7777777777777778,
   0.8666666666666667],
  'duration': 0.10933399200439453},
 'K-Nearest Neighbour': {'scores': [0.5666666666666667,
   0.6888888888888889,
   0.6111111111111112],
  'duration': 0.38782691955566406},
 'Naive Bayes': {'scores': [0.8333333333333334,
   0.8444444444444444,
   0.8777777777777778],
  'duration': 0.04604482650756836},
 'Quadratic Discriminant Analysis': {'scores': [0.7888888888888889,
   0.8111111111111111,
   0.8555555555555555],
  'duration': 0.05256295204162598},
 'Adaptive Boosting': {'scores': [0.7111111111111111,
   0.7555555555555555,
   0.8444444444444444],
  'duration': 0.24199175834655762},
 'Multilayer Perceptron': {'scores': [0.7555555555555555,
   0.7555555555555555,
   0.8666666666666667],
  'duration': 0.45665717124938965},
 'Deep Neural Network': {'scores': [0.5777

The 3 best performing classifiers are: Random Forest, Naive Bayes and Quadratic Discriminant Analysis. In the next section we are going to create an ensemble classifier out of those 3.

In [9]:
best_classifiers = [
    'Random Forest',
    'Naive Bayes',
    'Quadratic Discriminant Analysis'
]

## Assemble the Ensemble Classifier

In [10]:
def create_ensemble_estimator(verbose, random_state, n_jobs):
    from sklearn.ensemble import VotingClassifier
    estimators = list(
                map(
                    lambda name: (name, classifier_factories[name](verbose=verbose, random_state=RANDOM_SEED, n_jobs=-1)),
                    best_classifiers))

    return VotingClassifier(estimators, voting='soft')

print(f'Evaluating "Ensemble Classifier"')
t0 = time.time()
scores = evaluate_splits(create_ensemble_estimator)
duration = time.time() - t0
print(f'[Took {duration}s]\n')

scores

Evaluating "Ensemble Classifier"
[Took 2.948072910308838s]



[0.7888888888888889, 0.8333333333333334, 0.8888888888888888]

This classifier is now able to achive an accuracy of about 0.88, which is better than each single classifier.